[View in Colaboratory](https://colab.research.google.com/github/peter0083/van_ai_coding_challenge_3/blob/master/src/model.ipynb)

In [1]:
# first time using Google Colab
print("hello world!")

hello world!


In [2]:
import sys
sys.version
sys.version_info

sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)

In [3]:
# important dependencies
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy import stats
%matplotlib inline

In [4]:
# read csv
train_df = pd.read_csv('../data/train.csv')

In [5]:
train_df.shape

(550068, 12)

### 1. Preprocessing

In [6]:
train_df.head(10)

User_ID Product_ID Gender    Age  Occupation City_Category  \
0  1000001  P00069042      F   0-17          10             A   
1  1000001  P00248942      F   0-17          10             A   
2  1000001  P00087842      F   0-17          10             A   
3  1000001  P00085442      F   0-17          10             A   
4  1000002  P00285442      M    55+          16             C   
5  1000003  P00193542      M  26-35          15             A   
6  1000004  P00184942      M  46-50           7             B   
7  1000004  P00346142      M  46-50           7             B   
8  1000004   P0097242      M  46-50           7             B   
9  1000005  P00274942      M  26-35          20             A   

  Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \
0                          2               0                   3   
1                          2               0                   1   
2                          2               0                  12   
3                          2               0                  12   
4                         4+               0                   8   
5                          3               0                   1   
6                          2               1                   1   
7                          2               1                   1   
8                          2               1                   1   
9                          1               1                   8   

   Product_Category_2  Product_Category_3  Purchase  
0                 NaN                 NaN      8370  
1                 6.0                14.0     15200  
2                 NaN                 NaN      1422  
3                14.0                 NaN      1057  
4                 NaN                 NaN      7969  
5                 2.0                 NaN     15227  
6                 8.0                17.0     19215  
7                15.0                 NaN     15854  
8                16.0                 NaN     15686  
9                 NaN                 NaN      7871

In [7]:
# convert the discrete columns into dummy variables

dummy_cols = ['Product_ID', 'Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']
train_df_with_dummies = pd.get_dummies(train_df, columns= dummy_cols)

In [8]:
# inspect the train df with dummies

train_df_with_dummies.head(10)

User_ID  Occupation  Marital_Status  Product_Category_1  \
0  1000001          10               0                   3   
1  1000001          10               0                   1   
2  1000001          10               0                  12   
3  1000001          10               0                  12   
4  1000002          16               0                   8   
5  1000003          15               0                   1   
6  1000004           7               1                   1   
7  1000004           7               1                   1   
8  1000004           7               1                   1   
9  1000005          20               1                   8   

   Product_Category_2  Product_Category_3  Purchase  Product_ID_P00000142  \
0                 NaN                 NaN      8370                     0   
1                 6.0                14.0     15200                     0   
2                 NaN                 NaN      1422                     0   
3                14.0                 NaN      1057                     0   
4                 NaN                 NaN      7969                     0   
5                 2.0                 NaN     15227                     0   
6                 8.0                17.0     19215                     0   
7                15.0                 NaN     15854                     0   
8                16.0                 NaN     15686                     0   
9                 NaN                 NaN      7871                     0   

   Product_ID_P00000242  Product_ID_P00000342              ...                \
0                     0                     0              ...                 
1                     0                     0              ...                 
2                     0                     0              ...                 
3                     0                     0              ...                 
4                     0                     0              ...                 
5                     0                     0              ...                 
6                     0                     0              ...                 
7                     0                     0              ...                 
8                     0                     0              ...                 
9                     0                     0              ...                 

   Age_51-55  Age_55+  City_Category_A  City_Category_B  City_Category_C  \
0          0        0                1                0                0   
1          0        0                1                0                0   
2          0        0                1                0                0   
3          0        0                1                0                0   
4          0        1                0                0                1   
5          0        0                1                0                0   
6          0        0                0                1                0   
7          0        0                0                1                0   
8          0        0                0                1                0   
9          0        0                1                0                0   

   Stay_In_Current_City_Years_0  Stay_In_Current_City_Years_1  \
0                             0                             0   
1                             0                             0   
2                             0                             0   
3                             0                             0   
4                             0                             0   
5                             0                             0   
6                             0                             0   
7                             0                             0   
8                             0                             0   
9                             0                             1   

   Stay_In_Current_City_Years_2  Stay_

Now the dimensions are 550k x 3655. It is still good ratio. I am not running into the curse of dimensionality problem here because my observations are greater than 10 times the number of features. However, I should be careful not to overfit.

**Create X and Y arrays for actual training**

In [9]:
# create label

y = train_df_with_dummies['Purchase'].values
y.shape

(550068,)

In [10]:
# create features

X = train_df_with_dummies.drop(['Purchase'], axis=1).values
X.shape

(550068, 3654)

In [11]:
# create train-validation split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

In [12]:
X_train.shape

(440054, 3654)

In [13]:
X_val.shape

(110014, 3654)

### 2. Baseline Linear Regression

I'm not normalizing the features here yet because I am not doing feature selection.

In [16]:

baseline_lm = LinearRegression(fit_intercept=True, normalize=False)
baseline_lm.fit(X_train, y_train)

ValueError: could not convert string to float: 'P00208242'